In [1]:
pip install tensorflow==2.12.0

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement tensorflow==2.12.0 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0)
ERROR: No matching distribution found for tensorflow==2.12.0


In [3]:
pip install opencv-contrib-python

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install cvzone

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [11]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from cvzone.ClassificationModule import Classifier
import numpy as np
import math

In [13]:
cap = cv2.VideoCapture(0)
detector = HandDetector(maxHands=1)
classifier = Classifier("/Users/aashi/OneDrive/Desktop/sign/Model/keras_model.h5" , "/Users/aashi/OneDrive/Desktop/sign/Model/labels.txt")
offset = 20
imgSize = 300
counter = 0

labels = ["Hello","stop"]

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = '/Users/aashi/Desktop/sign/Model/keras_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [8]:
while True:
    success, img = cap.read()
    if not success:
        print("Failed to read from camera.")
        break

    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255

        # Clamp the bounding box coordinates
        height, width, _ = img.shape
        x1, y1 = max(0, x - offset), max(0, y - offset)
        x2, y2 = min(width, x + w + offset), min(height, y + h + offset)
        imgCrop = img[y1:y2, x1:x2]

        # Check if the crop is valid
        if imgCrop.size == 0:
            print("Empty crop detected! Skipping frame.")
            continue

        imgCropShape = imgCrop.shape
        aspectRatio = h / w

        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap: wCal + wGap] = imgResize
        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap: hCal + hGap, :] = imgResize

        # Get prediction
        prediction, index = classifier.getPrediction(imgWhite, draw=False)

        # Draw rectangles and labels
        cv2.rectangle(imgOutput, (x - offset, y - offset - 70), (x - offset + 400, y - offset + 10), (0, 255, 0), cv2.FILLED)
        cv2.putText(imgOutput, labels[index], (x, y - 30), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 2)
        cv2.rectangle(imgOutput, (x - offset, y - offset), (x + w + offset, y + h + offset), (0, 255, 0), 4)

        cv2.imshow('ImageCrop', imgCrop)
        cv2.imshow('ImageWhite', imgWhite)

    cv2.imshow('Image', imgOutput)
    cv2.waitKey(1)


NameError: name 'imgSize' is not defined